<a href="https://colab.research.google.com/github/spignelon/WordWeaver/blob/main/PEGASUS_Paraphraser.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! pip install sentence-splitter

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 1.7 MB/s eta 0:00:00


In [2]:
! pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 20.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 55.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 74.5 MB/s eta 0:00:00


In [3]:
! pip install SentencePiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.6 MB/s eta 0:00:00


In [7]:
# https://huggingface.co/tuner007/pegasus_paraphrase

import torch
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = 'tuner007/pegasus_paraphrase'
torch_device = 'cuda' if torch.cuda.is_available() else 'cpu'
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

def get_response(input_text,num_return_sequences):
  batch = tokenizer([input_text],truncation=True,padding='longest',max_length=60, return_tensors="pt").to(torch_device)
  translated = model.generate(**batch,max_length=60,num_beams=10, num_return_sequences=num_return_sequences, temperature=1.5)
  tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
  return tgt_text

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at tuner007/pegasus_paraphrase and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [8]:
text = "This announcement has caused a stir in the industry, with some users still investigating the degree to which they’ll be affected."

In [9]:
get_response(text, 5)

['Some users are still investigating the degree to which they will be affected by this announcement.',
 'Some users are still looking at the degree to which they will be affected by this announcement.',
 'Some users are still investigating the degree to which they will be affected by the announcement.',
 'Users are still investigating the degree to which they will be affected by this announcement.',
 'Some users are still investigating the degree to which they will be affected by this announcement, which has caused a stir in the industry.']

### Processing a paragraph of text

In [12]:
context = "An elephant's most obvious part is the trunk. The trunk is a very long nose, made from the upper lip. An elephant uses its trunk to grab objects such as food. Though the rest of an elephant's skin is strong and thick, its trunk is very soft and sensitive. Elephants avoid Acacia trees with ants because they can bite the inside of an elephant's trunk. The trunk is also used when it trumpets. The elephant usually stands still, raises its trunk, and blows. This is a signal to other elephants and animals."
print(context)

An elephant's most obvious part is the trunk. The trunk is a very long nose, made from the upper lip. An elephant uses its trunk to grab objects such as food. Though the rest of an elephant's skin is strong and thick, its trunk is very soft and sensitive. Elephants avoid Acacia trees with ants because they can bite the inside of an elephant's trunk. The trunk is also used when it trumpets. The elephant usually stands still, raises its trunk, and blows. This is a signal to other elephants and animals.


In [13]:
# Takes the input paragraph and splits it into a list of sentences
from sentence_splitter import SentenceSplitter, split_text_into_sentences

splitter = SentenceSplitter(language='en')

sentence_list = splitter.split(context)
sentence_list

["An elephant's most obvious part is the trunk.",
 'The trunk is a very long nose, made from the upper lip.',
 'An elephant uses its trunk to grab objects such as food.',
 "Though the rest of an elephant's skin is strong and thick, its trunk is very soft and sensitive.",
 "Elephants avoid Acacia trees with ants because they can bite the inside of an elephant's trunk.",
 'The trunk is also used when it trumpets.',
 'The elephant usually stands still, raises its trunk, and blows.',
 'This is a signal to other elephants and animals.']

In [14]:
# Do a for loop to iterate through the list of sentences and paraphrase each sentence in the iteration
paraphrase = []

for i in sentence_list:
  a = get_response(i,1)
  paraphrase.append(a)

In [15]:
paraphrase

[['The trunk is the most obvious part of an elephant.'],
 ["The trunk's nose is made from the upper lip."],
 ['An elephant has a trunk.'],
 ["The trunk of an elephant is very soft and sensitive, even though the rest of the elephant's skin is strong and thick."],
 ["Elephants don't like trees with ants because they can bite inside of an elephant's trunk."],
 ['When it trumpets, the trunk is also used.'],
 ['The elephant blows when it stands still.'],
 ['This is a sign of things to come.']]

In [16]:
paraphrase2 = [' '.join(x) for x in paraphrase]
paraphrase2

['The trunk is the most obvious part of an elephant.',
 "The trunk's nose is made from the upper lip.",
 'An elephant has a trunk.',
 "The trunk of an elephant is very soft and sensitive, even though the rest of the elephant's skin is strong and thick.",
 "Elephants don't like trees with ants because they can bite inside of an elephant's trunk.",
 'When it trumpets, the trunk is also used.',
 'The elephant blows when it stands still.',
 'This is a sign of things to come.']

In [19]:
paraphrase3 = [' '.join(x for x in paraphrase2) ]
paraphrased_text = str(paraphrase3).strip('[]').strip("'")
paraphrased_text

'"The trunk is the most obvious part of an elephant. The trunk\'s nose is made from the upper lip. An elephant has a trunk. The trunk of an elephant is very soft and sensitive, even though the rest of the elephant\'s skin is strong and thick. Elephants don\'t like trees with ants because they can bite inside of an elephant\'s trunk. When it trumpets, the trunk is also used. The elephant blows when it stands still. This is a sign of things to come."'

In [20]:
# Comparison of the original (context variable) and the paraphrased version (paraphrase3 variable)

print(context)
print(paraphrased_text)

An elephant's most obvious part is the trunk. The trunk is a very long nose, made from the upper lip. An elephant uses its trunk to grab objects such as food. Though the rest of an elephant's skin is strong and thick, its trunk is very soft and sensitive. Elephants avoid Acacia trees with ants because they can bite the inside of an elephant's trunk. The trunk is also used when it trumpets. The elephant usually stands still, raises its trunk, and blows. This is a signal to other elephants and animals.
"The trunk is the most obvious part of an elephant. The trunk's nose is made from the upper lip. An elephant has a trunk. The trunk of an elephant is very soft and sensitive, even though the rest of the elephant's skin is strong and thick. Elephants don't like trees with ants because they can bite inside of an elephant's trunk. When it trumpets, the trunk is also used. The elephant blows when it stands still. This is a sign of things to come."
